**Las funciones que se nos solicitan son:**

    ► Cantidad de películas por idioma

    ► Duración y año de la película ingresada

    ► Cantidad de películas por franquicia, ganancia total y promedio

    ► Cantidad de películas producidas por país

    ► Cantidad de películas por productora y revenue del mismo
    
    ► Nombre de director, éxito del mismo medido a través del retorno, nombre de cada película en la que participó con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.

In [91]:
import pandas as pd
import numpy as np
import unicodedata
from datetime import datetime
import ast
import re

In [92]:
df = pd.read_csv("Movies_EDA.csv", sep=",")

In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42195 entries, 0 to 42194
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               42195 non-null  int64  
 1   title            42195 non-null  object 
 2   overview         42195 non-null  object 
 3   release_date     42195 non-null  object 
 4   release_year     42195 non-null  int64  
 5   budget           42195 non-null  float64
 6   return           42195 non-null  float64
 7   popularity       42195 non-null  float64
 8   vote_average     42195 non-null  float64
 9   vote_count       42195 non-null  float64
 10  revenue          42195 non-null  float64
 11  runtime          42195 non-null  float64
 12  genres_names     42195 non-null  object 
 13  names_actors     42195 non-null  object 
 14  director         42195 non-null  object 
 15  companies_names  42195 non-null  object 
 16  name_collection  42195 non-null  object 
 17  countries_na

Eliminamos duplicados de la columna id.

In [94]:
df.drop_duplicates(inplace=True)
df.drop_duplicates(subset=['id'],inplace=True)
df=df.reset_index(drop=True)

In [95]:
df.drop(['id', 'overview', 'genres_names', 'popularity', 'vote_average', 'vote_count', 'names_actors'], axis=1, inplace=True)

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42195 entries, 0 to 42194
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            42195 non-null  object 
 1   release_date     42195 non-null  object 
 2   release_year     42195 non-null  int64  
 3   budget           42195 non-null  float64
 4   return           42195 non-null  float64
 5   revenue          42195 non-null  float64
 6   runtime          42195 non-null  float64
 7   director         42195 non-null  object 
 8   companies_names  42195 non-null  object 
 9   name_collection  42195 non-null  object 
 10  countries_names  42195 non-null  object 
 11  lenguages_names  42195 non-null  object 
dtypes: float64(4), int64(1), object(7)
memory usage: 3.9+ MB


API 1.

In [97]:
def idiomas_peliculas(idioma: str):
    idiomas = df[df['lenguages_names'].apply(lambda x: idioma in x)]
    total = len(idiomas)
    return f"Idioma: {idioma}\nTotal de películas {total}"


In [98]:
resultado = idiomas_peliculas("Español")
print(resultado)

Idioma: Español
Total de películas 2258


API 2.

In [99]:
def peliculas_duracion(pelicula: str):
    pelicula = pelicula.lower()
    resultado = df[df['title'].str.lower() == pelicula]
    if len(resultado) > 0:
        duracion = resultado['runtime'].values[0]
        año = resultado['release_year'].values[0]
        return f"Película: {pelicula}\nDuración: {duracion}\nAño: {año}"
    else:
        return f"No se encontró información para la película: {pelicula}"

In [100]:
resultado = peliculas_duracion("Titanic")
print(resultado)

Película: titanic
Duración: 194.0
Año: 1997


API 3.

In [101]:
def franquicia(Franquicia: str):
    franquicia_busqueda = Franquicia.lower()
    franquicia_filtrada = df[df['name_collection'].str.lower().str.contains(franquicia_busqueda)]
    
    cantidad_peliculas = len(franquicia_filtrada)
    ganancia_total = round(franquicia_filtrada['return'].sum(), 2)
    promedio_ganancia = round(franquicia_filtrada['return'].mean(), 2)
    
    return f"Franquicia: {Franquicia}\nCantidad de películas: {cantidad_peliculas}\nGanancia total: {ganancia_total}\nPromedio de ganancia: {promedio_ganancia}"

In [102]:
resultado = franquicia("Harry Potter")
print(resultado)

Franquicia: Harry Potter
Cantidad de películas: 8
Ganancia total: 53.17
Promedio de ganancia: 6.65


API 4.

In [103]:
def peliculas_pais(Pais: str):
    pais_busqueda = Pais.lower()
    peliculas_producidas = df[df['countries_names'].str.lower().str.contains(pais_busqueda)]
    cantidad_peliculas = len(peliculas_producidas)
    return f"País: {Pais}\nCantidad de películas producidas: {cantidad_peliculas}"

In [104]:
resultado = peliculas_pais("Canada")
print(resultado)

País: Canada
Cantidad de películas producidas: 1639


API 5.

In [105]:
def productoras_exitosas(Productora: str):
    productora_busqueda = Productora.lower()
    productora_filtrada = df[df['companies_names'].str.lower().str.contains(productora_busqueda)]
    
    cantidad_peliculas = len(productora_filtrada)
    revenue_total = productora_filtrada['revenue'].sum()
    
    return f"Productora: {Productora}\nCantidad de películas realizadas: {cantidad_peliculas}\nRevenue total: {revenue_total}"


In [106]:
resultado = productoras_exitosas("Paramount Pictures")
print(resultado)

Productora: Paramount Pictures
Cantidad de películas realizadas: 934
Revenue total: 44777586089.0


API 6.

In [107]:
def get_director(nombre_director):
    resultado = ""

    for index, row in df.iterrows():
        director = row['director']
        if isinstance(director, str) and nombre_director.lower() == director.lower():
            titulo = row['title']
            fecha_lanzamiento = pd.to_datetime(row['release_date']).date()
            retorno = round(row['return'], 2)
            costo = int(row['budget'])
            ganancia = int(row['revenue'])
            
            resultado += f"Película: {titulo}\nFecha de lanzamiento: {fecha_lanzamiento}\nRetorno: {retorno}\nCosto: {costo}\nGanancia: {ganancia}\n\n"
    
    if resultado == "":
        resultado = "No se encontró al director especificado."

    return resultado

In [108]:
nombre_director = "James Cameron"
resultado = get_director(nombre_director)
print(resultado)

Película: True Lies
Fecha de lanzamiento: 1994-07-14
Retorno: 3.29
Costo: 115000000
Ganancia: 378882411

Película: Terminator 2: Judgment Day
Fecha de lanzamiento: 1991-07-01
Retorno: 5.2
Costo: 100000000
Ganancia: 520000000

Película: The Abyss
Fecha de lanzamiento: 1989-08-09
Retorno: 1.29
Costo: 70000000
Ganancia: 90000098

Película: Aliens
Fecha de lanzamiento: 1986-07-18
Retorno: 9.91
Costo: 18500000
Ganancia: 183316455

Película: The Terminator
Fecha de lanzamiento: 1984-10-26
Retorno: 12.25
Costo: 6400000
Ganancia: 78371200

Película: Titanic
Fecha de lanzamiento: 1997-11-18
Retorno: 9.23
Costo: 200000000
Ganancia: 1845034188

Película: Piranha Part Two: The Spawning
Fecha de lanzamiento: 1981-01-01
Retorno: 0.0
Costo: 145786
Ganancia: 0

Película: Ghosts of the Abyss
Fecha de lanzamiento: 2003-04-10
Retorno: 1.31
Costo: 13000000
Ganancia: 17040871

Película: Aliens of the Deep
Fecha de lanzamiento: 2005-01-21
Retorno: 0.0
Costo: 0
Ganancia: 0

Película: Avatar
Fecha de lanzamie

Las funciones de la 2 a la 6 están normalizadas.